In [1]:
import numpy as np

Defining the Sigmoid function

\begin{align}
\sigma{(x)} & = 1/1+e^{-x}
\end{align}

In [3]:
def sigmoid(x):
    return 1/1+np.exp(-x)

Taking the derivative of sigmoid function we get
\begin{equation*}
\frac{\partial }{\partial x}  \sigma{(x)}  = {\sigma{(x)}} * (1-\sigma{(x)})
\end{equation*}

In [4]:
def sigmoid_prime(x):
    return sigmoid(x) * 1-(sigmoid(x))

In [5]:
# Input data
x = np.array([0.1, 0.3])
# Target
y = 0.2
# Input to output weights
weights = np.array([-0.8, 0.5])

In [6]:
learnrate = 0.5

In [7]:
# the linear combination performed by the node (h in f(h) and f'(h))
h = x[0]*weights[0] + x[1]*weights[1]

The output of neural network would be

In [8]:
nn_output = sigmoid(h)

 \begin{align}
 Error =  y - {y\widehat{}}
         \end{align}

Here,
\begin{equation*}
y\widehat{}
\end{equation*}
is actually nn_output

In [9]:
error = y - nn_output

Gradient of output can be found by passing the output(*h*) through the derivative of sigmoid

In [12]:
output_grad = sigmoid_prime(h)

 \begin{align}
 \delta =  Error * output grad
         \end{align}

In [13]:
error_term = error*output_grad

Now Let us perfrom Gradient Decent Step

In [14]:
del_w = [ learnrate * error_term * x[0],
          learnrate * error_term * x[1]]

In [15]:
print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

Neural Network output:
[1.90483742 1.74081822]
Amount of Error:
[1.90483742 1.74081822]
Change in Weights:
[array([0., 0.]), array([0., 0.])]


Now let us try to implement the same for 4 values

In [17]:
learnrate = 0.5
x = np.array([1, 2, 3, 4])
y = np.array(0.5)
# Initial weights
w = np.array([0.5, -0.5, 0.3, 0.1])

In [28]:
h = np.dot(x,w)

In [35]:
nn_output=sigmoid(h)

In [37]:
error = y - nn_output

In [38]:
output_grad = sigmoid_prime(h)


In [39]:
error_term = error * output_grad


In [40]:
del_w = learnrate * error_term * x


In [41]:
print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

Neural Network output:
1.4493289641172216
Amount of Error:
-0.9493289641172216
Change in Weights:
[-0. -0. -0. -0.]
